<h1 align='center'> Glean CV Approach </h1>
We are going to see how we can try Computer Vision Approach to Classify various vendors. The idea is to use EfficientNetB0 to train on the images of first pages of all pdf documents.

In [1]:
from utilities import *
from PIL import Image
import tensorflow as tf
from keras.applications.efficientnet import EfficientNetB0
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
import pprint


<h3> Getting Dataset </h3>

Initially our files are present as pdf we are going to conver them into images. We are taking the  first page of each pdf file and using it to get the image from that page and going to train only on those  

In [2]:
#path_of_dataset=input('Enter Path of Dataset to Begin: ')
X,Y,map=prepare_all_dataset('../Dataset')#path_of_dataset)
n_class=len(map)
X=get_images_array(X)
Y=np.asarray(Y)

dump_load_map(map,0)
#print(Y)

Y=tf.keras.utils.to_categorical(
    Y, num_classes=n_class, dtype='float32')


class_weights=get_class_weights(Y)
X_train,X_test,Y_train,Y_test=train_test_split(X,Y)


Here are vendors: ['../Dataset/ADP_LLC', '../Dataset/Align_Technology_Inc', '../Dataset/ALLIANCE_VANTAGE_LIMITED', '../Dataset/Amazon_Business', '../Dataset/Apple_Inc (invoices)', '../Dataset/apple_search_ads', '../Dataset/Atlassian', '../Dataset/Avenue_Logistics', '../Dataset/Aws', '../Dataset/AXLE_LOGISTICS', '../Dataset/Bacon_Work', '../Dataset/CDW_Direct', '../Dataset/CLEARFREIGHT_INC', '../Dataset/Cooley_LLP', '../Dataset/CSC', '../Dataset/Datadog', '../Dataset/DHL_eCommerce', '../Dataset/Electric_Al_Inc', '../Dataset/Employee_Benefits_Corporation', '../Dataset/Equifax_Information_SVCS_LLC', '../Dataset/esendex', '../Dataset/Facebook_Inc', '../Dataset/FedEx', '../Dataset/Freshworks_Inc_', '../Dataset/Github', '../Dataset/Glean_Analytics_Inc', '../Dataset/Goodwin_Procter_LLP', '../Dataset/google_llc', '../Dataset/Gowling_WLG_(Canada)_LLP', '../Dataset/Hubspot', '../Dataset/Lindenmeyr_Munroe', '../Dataset/LinkedIn_Corporation', '../Dataset/MainStreet_Work_Inc', '../Dataset/Maples_an

dirs: 100%|██████████| 97/97 [00:00<00:00, 135.55it/s]



Align_Technology_Inc vendor Processing 


dirs: 100%|██████████| 51/51 [00:00<00:00, 117.57it/s]



ALLIANCE_VANTAGE_LIMITED vendor Processing 


...

dirs: 100%|██████████| 101/101 [00:00<00:00, 104.18it/s]



WeWork_Companies_Inc_ vendor Processing 


dirs: 100%|██████████| 35/35 [00:00<00:00, 138.31it/s]



zoom vendor Processing 


dirs: 100%|██████████| 36/36 [00:00<00:00, 134.32it/s]


In [3]:
pprint.pprint(class_weights)

{0: 0.8906122448979592,
 1: 1.5585714285714285,
 2: 2.182,
 3: 0.2909333333333333,
 4: 0.9091666666666667,
 5: 0.8906122448979592,
 6: 4.364,
 7: 1.5048275862068965,
 8: 0.2346236559139785,
 9: 0.8728,
 10: 0.8728,
 11: 0.6234285714285714,
 12: 1.6784615384615384,
 13: 0.8728,
 14: 1.6784615384615384,
 15: 1.9836363636363636,
 16: 0.9091666666666667,
 17: 0.9091666666666667,
 18: 1.7456,
 19: 1.407741935483871,
 20: 3.117142857142857,
 21: 5.455,
 22: 0.7273333333333334,
 23: 2.4244444444444446,
 24: 1.118974358974359,
 25: 1.2835294117647058,
 26: 0.8392307692307692,
 27: 0.9697777777777777,
 28: 0.9285106382978724,
 29: 0.5594871794871795,
 30: 1.407741935483871,
 31: 0.9918181818181818,
 32: 1.148421052631579,
 33: 0.855686274509804,
 34: 0.8728,
 35: 1.3224242424242425,
 36: 0.9285106382978724,
 37: 1.3224242424242425,
 38: 0.8728,
 39: 0.8728,
 40: 0.6713846153846154,
 41: 1.7456,
 42: 2.182,
 43: 1.7456,
 44: 0.8728,
 45: 0.8728,
 46: 1.6784615384615384,
 47: 0.8728,
 48: 2.56705

<h3> Creating Architecture of EffiicnetNetB0 </h3>

In [4]:
def Custom_Efficient_Net(image_shape=X[0].shape,output_shape=n_class):
    if 'Efficient_net_custom.h5' in os.listdir(os.getcwd()):
        print('Loaded Previous Model ')
        model=tf.keras.models.load_model('Efficient_net_custom.h5')
    else: 
        input_val=tf.keras.layers.Input(shape=image_shape)
        model=EfficientNetB0(include_top=False,input_tensor=input_val,weights='imagenet')

        for i in range(int(len(model.layers) * 0.75)):
            model.layers[i].trainable = False

        #model.trainable = False


        x = tf.keras.layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
        x = tf.keras.layers.BatchNormalization()(x)

        top_dropout_rate = 0.8
        x = tf.keras.layers.Dropout(top_dropout_rate, name="top_dropout")(x)
        #x=tf.keras.layers.Flatten()
        output = tf.keras.layers.Dense(output_shape, activation="softmax", name="pred")(x)

        model=tf.keras.Model(inputs=input_val,outputs=output)
    
    model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

custom_model=Custom_Efficient_Net()

<h3> Training The Model </h3>

In [5]:
callback = tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss',mode='auto',restore_best_weights=True )
hist=custom_model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=25,callbacks=[callback],batch_size=128,class_weight=class_weights)

Epoch 1/25
13/13 [==============================] - 41s 3s/step - loss: 1.2742 - accuracy: 0.7506 - val_loss: 3.3825 - val_accuracy: 0.2143
Epoch 2/25
 9/13 [===================>..........] - ETA: 10s - loss: 0.0791 - accuracy: 0.9835

KeyboardInterrupt: 

<h3> Saving Model and plotting the graphs </h3>

In [ ]:
custom_model.save('Efficient_net_custom.h5')
plot_metrics_for_training(hist.history)

In [ ]:
Yverify=custom_model.predict(X_test)
Yverify=np.argmax(Yverify,axis=1).reshape(-1,1)
Yverify_ground_truth=np.argmax(Y_test,axis=1).reshape(-1,1)
op=classification_report(Yverify_ground_truth,Yverify)
print(op)

<h3> Inference testing </h3>



In [ ]:
#D:\Fuse_Work\Glean_Experiments\Dataset\Medline\01GH7C1RYM86F7JDQFJ1944V84.pdf
loaded_model=tf.keras.models.load_model('Efficient_net_custom.h5')
map=dump_load_map({},1)
path_image=input('Enter PDF Path: ')
test_img=inference_image_get(path_image)
preds=loaded_model.predict(test_img.reshape(1,test_img.shape[0],test_img.shape[1],test_img.shape[2]))
op=np.argmax(preds)
print(list(preds[0])[op])
if list(preds[0])[op]<=0.6:
    print('This file belongs to none of the vendors ')
else: 
    print(list(filter(lambda x: map[x] == op, map))[0])